In [ ]:
# 리사이즈 - 나만
import os
from PIL import Image, ImageOps

input_folder = "D:\\foodsnap_CNN\\건강관리를 위한 음식 이미지\\Training\\train"
output_folder = "D:\\foodsnap_CNN\\건강관리를 위한 음식 이미지\\Training\\train_p" #리사이즈 train
valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')

def pad_and_resize_save(img_path, save_path, size=(224, 224)):
    try:
        img = Image.open(img_path)
        img_padded = ImageOps.pad(img, size, color=(0, 0, 0))
        img_padded.save(save_path)
        img.close()
        os.remove(img_path)  # 원본 삭제
    except Exception as e:
        print(f"❌ 에러 발생: {img_path} → {e}")

# 전체 이미지 순회
for root, dirs, files in os.walk(input_folder):
    rel_path = os.path.relpath(root, input_folder)
    save_dir = os.path.join(output_folder, rel_path)
    os.makedirs(save_dir, exist_ok=True)
    for file in files:
        if file.lower().endswith(valid_extensions):
            input_path = os.path.join(root, file)
            output_path = os.path.join(save_dir, file)
            pad_and_resize_save(input_path, output_path)

print(f'✅ padding + resize + 원본 삭제 완료 → 저장 위치: {output_folder}')

In [ ]:
CHUNK_START = 1   # 내가 맡은 첫 청크
CHUNK_END   = 10  # 내가 맡은 마지막 청크
# 10단위 / A 1-10 / B 11-20 / A 21-30 / B 31-40.....
## 계속 범위 바꿔가며 실행

In [ ]:
import os, gc, tensorflow as tf
from pathlib import Path
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau,
                                        ModelCheckpoint, LearningRateScheduler)

# ────────────────────── 설정 ──────────────────────
CHUNKS_ROOT  = Path(r"D:\\foodsnap_CNN\\건강관리를 위한 음식 이미지\\Training\\chunks")
CKPT_DIR     = CHUNKS_ROOT.parent / "ckpt"      # ckpt 저장 폴더(Training/ckpt)
CKPT_DIR.mkdir(exist_ok=True)

IMG_SIZE     = (224, 224)
BATCH_SIZE   = 64
EPOCHS_PER_CHUNK = 15          # 청크당 epoch
UNFREEZE_STEP    = 40          # 청크가 넘어갈 때마다 뒤쪽 N레이어 더 풀기

# ───────────────── 데이터 제너레이터 공통 ─────────────────
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
    rotation_range=30, width_shift_range=.25, height_shift_range=.25,
    shear_range=.25, zoom_range=.25, horizontal_flip=True, vertical_flip=True,
    brightness_range=[.8,1.2], validation_split=.1)

def make_generators(chunk_dir:Path):
    train_gen = datagen.flow_from_directory(
        chunk_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', subset='training')
    val_gen   = datagen.flow_from_directory(
        chunk_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', subset='validation', shuffle=False)
    return train_gen, val_gen

# ───────────────── 모델 빌더 ─────────────────
def build_model(num_classes:int, unfreeze_from:int=None):
    base = MobileNetV3Large(input_shape=(*IMG_SIZE,3),
                            include_top=False, weights='imagenet')
    if unfreeze_from is not None:
        for l in base.layers[:unfreeze_from]:  l.trainable = False
        for l in base.layers[unfreeze_from:]: l.trainable = True
    else:   # 첫 청크: 헤드만 학습
        for l in base.layers: l.trainable = False

    x = GlobalAveragePooling2D()(base.output)
    x = Dense(512, activation='relu')(x); x = Dropout(.3)(x)
    x = Dense(256, activation='relu')(x)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(base.input, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ───────────────── 러닝레이트 스케줄러 ─────────────────
lr_sched = lambda e, lr: 1e-4 if e < 5 else 1e-5 if e < 10 else 5e-6

# ───────────────── 청크 순차 학습 루프 ─────────────────
if CHUNK_START > 1:
    latest_ckpt = CKPT_DIR / f"train_chunk{CHUNK_START-1}.keras"
else:
    latest_ckpt = None

for idx, chunk_dir in enumerate(sorted(CHUNKS_ROOT.glob("train_chunk*")), start=1):
    if not (CHUNK_START <= idx <= CHUNK_END):
        continue  # 내 작업 범위 아니면 skip

    print(f"\n▶️  학습 시작: {chunk_dir.name}")

    # ▶️ Epoch 동적 결정
    if idx <= 5:
        epochs = 20 # 기초 학습 강화
    elif idx <= 30:
        epochs = 15
    else:
        epochs = 20 # 파인튜닝 강화

    train_gen, val_gen = make_generators(chunk_dir)

    # ── 모델 준비 ──
    if latest_ckpt is None:          # 첫 청크
        model = build_model(train_gen.num_classes, unfreeze_from=None)
    else:                            # 이후 청크: 더 많은 레이어 풀어 fine-tune
        model = build_model(train_gen.num_classes, unfreeze_from=-UNFREEZE_STEP)
        model.load_weights(latest_ckpt)

    # ── 콜백 ──
    ckpt_path = CKPT_DIR / f"{chunk_dir.name}.keras"
    cbs = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=.2, patience=3),
        LearningRateScheduler(lr_sched),
        ModelCheckpoint(str(ckpt_path), save_best_only=True, monitor='val_accuracy', verbose=1)
    ]

    # ── 학습 ──
    print(f"📌 Epoch 설정: {epochs}회")
    model.fit(train_gen, validation_data=val_gen, epochs=epochs, callbacks=cbs)

    latest_ckpt = ckpt_path
    del train_gen, val_gen, model
    tf.keras.backend.clear_session()
    gc.collect()

print(f"\n🎉 모든 청크 학습 완료! 최종 모델: {latest_ckpt}")
